In [ ]:
!gdown '1fuPqJy8j8O_Xm2a46aenKlr31I_GGwXV'

Downloading...
From: https://drive.google.com/uc?id=1fuPqJy8j8O_Xm2a46aenKlr31I_GGwXV
To: /content/data_clean_v2.csv
100% 1.75M/1.75M [00:00<00:00, 52.3MB/s]


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'data_clean_v2.csv'
data = pd.read_csv(file_path)

# Displaying the first few rows of the dataset to understand its structure
data.head()


,index,location,companyType,decription,salaryType,contractType,jobType,experience,gender,vacacies,education,minAge,maxAge,skill,Salary,Partner
0,0,Tp Hồ Chí Minh,Công ty,MÔ TẢ CÔNG VIỆC\n🙋‍♂️ Bạn cần làm:\n- Nhận hàn...,Theo tháng,Bán thời gian,Tài xế giao hàng xe máy,NaN,Không yêu cầu,50,NaN,18,55.0,Không,9500000,Có
1,1,Hà Nội,Công ty,🔥🔥🔥 NINJA VAN - TUYỂN NHÂN VIÊN GIAO HÀNG KHU ...,Theo tháng,Toàn thời gian,Tài xế giao hàng xe máy,NaN,Không yêu cầu,5,NaN,18,40.0,Không,11500000,Có
2,2,Bình Dương,Công ty,"- Tìm kiếm khách hàng tiềm năng, xác định nhu ...",Theo tháng,Toàn thời gian,Nhân viên kinh doanh,1-2 năm,Không yêu cầu,2,Trung cấp/Nghề,22,35.0,Không,10500000,Có
3,3,Hà Nội,Công ty,"1. CÔNG VIỆC:\n- Sắp xếp, nhập xuất, phân chia...",Theo tháng,Toàn thời gian,Nhân viên kho vận,NaN,Không yêu cầu,10,NaN,18,35.0,Không,11000000,Có
4,4,Tp Hồ Chí Minh,Công ty,"Việc làm thời vụ lương tuần ca sáng, ca chiều....",Theo ngày,Thời vụ,Nhân viên kho vận,Không yêu cầu,Nam,20,NaN,18,48.0,Không,258000,Có


In [ ]:
missing_values = data.isna().sum()
missing_values[missing_values > 0]

experience    633
education     628
maxAge        202
dtype: int64

In [ ]:
grouped_data = data.groupby('jobType')

In [ ]:
import os

# Tạo thư mục nếu chưa tồn tại
output_directory = 'data'
os.makedirs(output_directory, exist_ok=True)

# Lặp qua từng giá trị unique trong 'jobType'
for job_type, group in grouped_data:
    # Đường dẫn tới file CSV đầu ra
    output_path = os.path.join(output_directory, f'data_{job_type}.csv')

    # Lưu group vào file CSV
    group.to_csv(output_path, index=False)

In [ ]:
from sklearn.impute import KNNImputer
import math
from sklearn.preprocessing import LabelEncoder

def encode_row(row, encoders, original_values):
    for column in row.index:
        if pd.notna(row[column]):
            # Chỉ mã hóa nếu giá trị không phải NaN
            if column in encoders:
                label_encoder = encoders[column]
                row[column] = label_encoder.transform([row[column]])[0]
                if column not in original_values:
                    original_values[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    return row


In [ ]:
imputer = KNNImputer(n_neighbors = 2)
# Đường dẫn đến thư mục chứa các DataFrame
folder_path = '/content/data'
out_path = 'data_fill'
os.makedirs(out_path, exist_ok=True)
# Lặp qua tất cả các tệp trong thư mục
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Kiểm tra xem có phải là tệp CSV không
        file_path = os.path.join(folder_path, file_name)

        # Đọc DataFrame từ tệp
        data = pd.read_csv(file_path)
        print(file_path)
        if data['maxAge'].isnull().sum() == len(data):
          data['maxAge'] = 60
        data.drop(columns = ['index'], inplace = True)
        df = data.drop(columns=['Salary','decription']).copy()
        df.reset_index(inplace = True)
        original_values = {}
        columns_to_encode = df.select_dtypes(include = 'object').columns
        columns = df.columns

        df_encoded = df.copy()
        encoders = {}
        original_values = {}
        for column in columns_to_encode:
          encoders[column] = LabelEncoder()
          encoders[column].fit(df[column].dropna().unique())
        df_encoded = df_encoded.apply(lambda row: encode_row(row, encoders, original_values), axis=1)

        df_encoded = imputer.fit_transform(df_encoded)
        print(columns)
        df_encoded = pd.DataFrame(df_encoded, columns = columns)
        for column in columns_to_encode:
          df_encoded[column] = df_encoded[column].astype(int).map({v: k for k, v in original_values[column].items()})


        df_encoded['maxAge'] = df_encoded['maxAge'].apply(math.ceil)
        df_encoded['decription'] = data['decription'].copy()
        df_encoded['Salary'] = data['Salary'].copy()
        # Lưu trữ DataFrame đã được xử lý vào một tệp mới
        output_file_path = os.path.join(out_path, f'processed_{file_name}')
        df_encoded.to_csv(output_file_path, index=False)
        print("Done!")


/content/data/data_Telesales.csv
Index(['index', 'location', 'companyType', 'salaryType', 'contractType',
       'jobType', 'experience', 'gender', 'vacacies', 'education', 'minAge',
       'maxAge', 'skill', 'Partner'],
      dtype='object')
Done!
/content/data/data_Thư ký.csv
Index(['index', 'location', 'companyType', 'salaryType', 'contractType',
       'jobType', 'experience', 'gender', 'vacacies', 'education', 'minAge',
       'maxAge', 'skill', 'Partner'],
      dtype='object')
Done!
/content/data/data_Giúp việc.csv
Index(['index', 'location', 'companyType', 'salaryType', 'contractType',
       'jobType', 'experience', 'gender', 'vacacies', 'education', 'minAge',
       'maxAge', 'skill', 'Partner'],
      dtype='object')
Done!
/content/data/data_Thợ cắt tóc.csv
Index(['index', 'location', 'companyType', 'salaryType', 'contractType',
       'jobType', 'experience', 'gender', 'vacacies', 'education', 'minAge',
       'maxAge', 'skill', 'Partner'],
      dtype='object')
Done!
/cont

In [ ]:
from sklearn.utils import shuffle

folder_path = '/content/data_fill'
merged_file_path = 'Data_KNN_fill.csv'

# Đọc tất cả các tệp trong thư mục 'data_fill' và merge chúng thành một DataFrame
merged_data = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.csv')])

# Xáo trộn các dòng trong DataFrame
shuffled_data = shuffle(merged_data)

# Lưu DataFrame đã xáo trộn thành một tệp mới
shuffled_data.to_csv(merged_file_path, index=False)